In [ ]:
def plot(ax,energy,tag=' ',line='-'):
    ax.set_prop_cycle(None)
    ax.plot(energy.t, energy.ke, label='ke '+tag, linestyle=line)
    ax.plot(energy.t, energy.gpe, label='gpe '+tag, linestyle=line)
    ax.plot(energy.t, energy.total/2, label='total/2 '+tag, linestyle=line)
def plot_difference(ax,energy1,energy2):
    ax.plot(energy1.t, energy1.ke-energy2.ke, label='ke')
#     ax.plot(energy1.t, energy1.gpe-energy2.ke, label='gpe')
#     ax.plot(energy1.t, energy1.total/2-energy2.total/2, label='total/2')  

from scipy.signal import argrelextrema
def fit(ax,energy,localmax=1):
    global T
    t = np.array(energy.t); total = np.array(energy.total)/2
    if localmax:
        localmax_index = argrelextrema(total, np.greater)[0]
        localmax_time = t[localmax_index]
        localmax = total[localmax_index]
        ax.plot(localmax_time,localmax,'.')
        a, b = np.polyfit(localmax_time, np.log(localmax), 1)  
    else:
        a, b = np.polyfit(t, np.log(total), 1)
    ax.plot(t, np.exp(b)*np.exp(a*t), linestyle='-.', label = 'a = %g' %(a/T))
    return (a/T)  

def read(name, skip = None):
    energy,exists = readin(path+name, table_delimiter = ' ', 
                       table_headers = ['t', 'ke', 'gpe'], skipn = skip)
    if exists:
        total = energy.ke + energy.gpe 
    energy.insert(len(energy.columns), "total", total, True)
    return energy
def read_ns(name, skip = 100):
    energy,exists = readin(path+name, table_delimiter = ' ', 
                       table_headers = ['t', 'ke', 'gpe', 'dissipation'], skipn = skip)
    if exists:
        total = energy.ke + energy.gpe 
    energy.insert(len(energy.columns), "total", total, True)
    return energy


In [ ]:
""" Energy dissipation for both cases. """
fig = plt.figure(figsize=[3,3]); ax = fig.gca()
tag = ''
kp = 2*np.pi/(200/5)
omegap = (9.8*kp)**0.5

path='/projects/DEIKE/jiarongw/multilayer/Ifremer/field_new_200m_P0.005_RE40000_10_15_rand2_Htheta0.503/'
energy = read('energy_before_remap.dat')
E0 = energy.total[0]
ax.plot(energy.t*omegap, energy.total,'-', label=r'$P=0.005$', lw=0.5)
# arrowprops=dict(color='C0', headwidth=2, headlength=5, width=0.2)
# path='/projects/DEIKE/jiarongw/multilayer/Ifremer/field_new_200m_P0.005_RE40000_10_15_rand2_Htheta0.503/restart/'
# energy = read('energy_before_remap.dat')
# ax.plot(energy.t*omegap, energy.total/E0,'-', label=r'$P=0.005$', lw=0.5, c='C0')

path='/projects/DEIKE/jiarongw/multilayer/revision/field_new_200m_P0.02_RE40000_10_15_rand4_Htheta0.503/'
energy = read('energy_before_remap.dat')
print(energy.total[0])
# ax.plot(energy.t*omegap, 2*energy.ke/energy.total[0],'--', lw=0.5)
# ax.plot(energy.t*omegap, 2*energy.gpe/energy.total[0],':', lw=0.5)
ax.plot(energy.t*omegap, energy.total,'-', label=r'$P=0.02$', lw=0.5)
arrowprops=dict(color='C0', headwidth=2, headlength=5, width=0.2)

# ax.set_ylim([0.2,1.1])
ax.set_xlim([0.,1000])
plt.legend(fancybox=False, fontsize=6, handlelength=1.1, labelspacing=0.4, handletextpad=0.2)
# ax.set_xticks([0,10,20,30,40,50])
plt.xlabel(r'$\omega_p t$')
plt.ylabel(r'$E/E_0$')
ax.tick_params(which='both', labelsize=8)

In [ ]:
path='/projects/DEIKE/jiarongw/multilayer/revision/field_new_200m_P0.005_RE40000_10_15_rand2_Htheta0.503/'
fig = plt.figure(layout='constrained', figsize=(10, 4))
fig, axes = plt.subplots(3, 4)

stats_t = []
for i,t in enumerate((0,200,400,600)):
    filename = path + 'surface/eta_matrix_%g' %t
    eta = np.fromfile(filename, dtype=np.float32)
    N = 1024; L0=200
    dx = L0/N
    eta = eta.reshape(N+1,N+1); eta = eta[1:,1:]
    axes[0,i].imshow(np.rot90(eta), cmap='RdBu_r')
    detadx = np.gradient(eta,axis=0)/dx
    axes[1,i].imshow(np.rot90(detadx), cmap='RdBu_r')
    detady = np.gradient(eta,axis=1)/dx
    axes[2,i].imshow(np.rot90(detady), cmap='RdBu_r')
    axes[0,i].axis('off')
    axes[1,i].axis('off')
    axes[2,i].axis('off')
    stats = [np.ravel(eta), np.ravel(detadx), np.ravel(detady)]
    stats_t.append(stats)

In [ ]:
from scipy.stats import norm

for it in range(0,4):
    slopex_bins = np.linspace(-1, 1, num=60, endpoint=True) # wave elevation bins in the unit of m 
    pdf, bins = np.histogram(stats_t[it][0], bins=slopex_bins, density=True)
    bin_width = bins[1:] - bins[:-1] 
    bin_centers = (bins[:-1] + bin_width/2)
    stdslope = np.std(stats_t[it][0])
    print(stdslope)
    plt.plot(bin_centers, pdf, label='t=%g' %(it*200), c=plt.get_cmap('Blues')((it+1)/6))
    
# plt.plot(bin_centers, normpdf(bin_centers, 0, np.std(stats_t[0][0])), c='k')
plt.legend()
# plt.yscale('log')

### Horizontal slope distribution 
Compute the horizontal slope distribution and compare with the Choppy formula.

In [ ]:
from scipy.special import erf

def choppy(x, sigma):
    pdf = (np.exp(-0.5*sigma**2)/np.pi/(1+x**2)**2 + 1/(2*np.pi*sigma**2)**0.5*(sigma**2*(1+x**2)+1)/(1+x**2)**2.5) * erf(1/(2*sigma**2*(1+x**2))**0.5) * np.exp(-1/2/sigma**2*x**2/(1+x**2))
    return pdf

def normpdf(x, mean, sd):
    var = sd**2
    denom = (2*np.pi*var)**.5
    num = np.exp(-(x-mean)**2/(2*var))
    return num/denom

from scipy.stats import norm

for it in range(0,4):
    slopex_bins = np.linspace(-0.2, 0.2, num=60, endpoint=True) # wave elevation bins in the unit of m 
    pdf, bins = np.histogram(stats_t[it][1], bins=slopex_bins, density=True)
    bin_width = bins[1:] - bins[:-1] 
    bin_centers = (bins[:-1] + bin_width/2)
    stdslope = np.std(stats_t[it][1])
    print(stdslope)
    plt.plot(bin_centers, pdf, label='$t=%g, \sigma=%.3f$' %(it*200, stdslope), c=plt.get_cmap('Blues')((it+1)/6))
    plt.plot(bin_centers, choppy(bin_centers, stdslope), '--', label='choppy, $\sigma=%.3f$' %stdslope, c=plt.get_cmap('Blues')((it+1)/6))
    plt.plot(bin_centers, normpdf(bin_centers, 0, stdslope), '.-', label='gaussian, $\sigma=%.3f$' %stdslope, c=plt.get_cmap('Blues')((it+1)/6))
    
# plt.plot(bin_centers, normpdf(bin_centers, 0, np.std(stats_t[0][1])), c='k')
# plt.ylim
plt.legend()
plt.yscale('log')
plt.xlabel('$\partial \eta/\partial x$')
plt.ylabel('pdf')

In [ ]:
from scipy.stats import norm

for it in range(0,4):
    slopex_bins = np.linspace(-0.2, 0.2, num=30, endpoint=True) # wave elevation bins in the unit of m 
    pdf, bins = np.histogram(stats_t[it][1], bins=slopex_bins, density=True)
    bin_width = bins[1:] - bins[:-1] 
    bin_centers = (bins[:-1] + bin_width/2)
    stdslope = np.std(stats_t[it][1])
    print(stdslope)
    plt.plot(bin_centers/stdslope, pdf*stdslope, label='t=%g' %(it*200), c=plt.get_cmap('Blues')((it+1)/6))
    
# plt.plot(bin_centers, normpdf(bin_centers, 0, np.std(stats_t[0][1])), c='k')
plt.legend()
plt.yscale('log')
